# **TensorFlow Stellar Spectra**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Deyht/AI_astro_ED_AAIF/blob/main/practical_works/CNN/classification/spectra/Spectra_TensorFlow.ipynb)

---


**Link to the CIANNA github repository**
https://github.com/Deyht/CIANNA

In [ ]:
%cd /content/

import numpy as np
import matplotlib.pyplot as plt
import os

import sys, glob

from tensorflow import keras
from tensorflow.keras import layers
from sklearn import metrics
import tensorflow as tf

############################################################################
##              Data reading (your mileage may vary)
############################################################################


if(not os.path.isdir("stellar_spectra_data")):
		os.system("wget https://share.obspm.fr/s/ANxKkxAZoKmXzRw/download/stellar_spectra_data.tar.gz")
		os.system("tar -xzf stellar_spectra_data.tar.gz")

print ("Reading inputs ... ", end = "", flush=True)

nb_class = 7
raw_spectra_size = 3753
spectra_size = 3776
nb_spectra = 1115
nb_keep_val = 200

class_balance = [10,30,30,20,20,30,10]
nb_train = np.sum(class_balance) # rebalanced over 1115

######################### ##########################
#          Loading data and pre process
######################### ##########################

raw_data = np.loadtxt("stellar_spectra_data/train.dat")
raw_target = np.loadtxt("stellar_spectra_data/target.dat")

input = np.tanh(40*np.clip(raw_data,0.,1.))
targ = raw_target

id_classes = []
for i in range(0, nb_class):
	temp = np.where(np.argmax(targ[:,:], axis=1) == i)
	id_classes.append(temp)

input[:,:] -= np.mean(input[:,:], axis = 0)
input[:,:] /= np.max(np.abs(input[:,:]), axis = 0)

# split training and test dataset
input_test = input[-nb_keep_val:,:]
targ_test = targ[-nb_keep_val:,:]

print (np.shape(input_test))

input_train = np.empty((0,raw_spectra_size))
targ_train = np.empty((0,nb_class))

for i in range(0,nb_class):
	index_list = id_classes[i][0][:class_balance[i]]
	for j in range(0, len(index_list)):
		input_train = np.append(input_train, np.reshape(input[index_list[j],:],(1,raw_spectra_size)), axis=0)
		targ_train = np.append(targ_train, np.reshape(targ[index_list[j],:],(1,nb_class)), axis=0)

pad_in_train = np.zeros((nb_train, spectra_size))
pad_in_test = np.zeros((nb_keep_val, spectra_size))

pre_pad = np.maximum(0,(spectra_size - raw_spectra_size)//2)
post_pad = np.maximum(0,(spectra_size - raw_spectra_size)//2 + (spectra_size - raw_spectra_size)%2)

pad_in_train[:,pre_pad:-post_pad] = input_train[:,:]
pad_in_test[:,pre_pad:-post_pad] = input_test[:,:]

#Reshape to allow initial padding in the model
pad_in_train = np.reshape(pad_in_train, (nb_train,spectra_size,1))
pad_in_test = np.reshape(pad_in_test, (nb_keep_val,spectra_size,1))

model = keras.Sequential()
model.add(keras.layers.ZeroPadding1D(padding=2))
model.add(layers.Conv1D(filters=8 , kernel_size=8, strides=4, activation='relu'))
model.add(keras.layers.ZeroPadding1D(padding=2))
model.add(layers.Conv1D(filters=16, kernel_size=8, strides=4, activation='relu'))
model.add(keras.layers.ZeroPadding1D(padding=2))
model.add(layers.Conv1D(filters=32, kernel_size=8, strides=4, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(units=256))
model.add(layers.Dropout(0.4))
model.add(layers.Activation('relu'))
model.add(layers.Dense(units=256))
model.add(layers.Dropout(0.4))
model.add(layers.Activation('relu'))
model.add(layers.Dense(units=nb_class, activation='softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

for run_iter in range(0,4):

	model.fit(pad_in_train, targ_train, batch_size=16, epochs=50, shuffle=True, validation_data=(pad_in_test, targ_test))

	pred = model.predict(pad_in_test)

	matrix = metrics.confusion_matrix(targ_test.argmax(axis=1), pred.argmax(axis=1))
	print (matrix)

